In [1]:
%matplotlib widget

In [2]:
import matplotlib as mpl
from matplotlib.gridspec import GridSpec

import pickle

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import scipy
from scipy import stats
# import seaborn as sns

import ipywidgets
from ipywidgets import widgets
import qgrid
# from IPython.display import display


import os
import mplcursors
# import re

import nmrProblem_28122020 as nmrProblem

import nmrglue as ng

import tempfile
import io

In [3]:
print("matplolib:", mpl.__version__)
print("numpy:", np.__version__)
print("pandas:", pd.__version__)
print("scipy:", scipy.__version__)
print("ipywidgets:", ipywidgets.__version__)
print("qgrid:", qgrid.__version__)
print("mplcursors:", mplcursors.__version__)
print("nmrglue:", ng.__version__)


matplolib: 3.3.2
numpy: 1.19.2
pandas: 1.1.3
scipy: 1.5.2
ipywidgets: 7.5.1
qgrid: 1.3.1
mplcursors: 0.4
nmrglue: 0.8


In [4]:
qgrid_opts = {
    # SlickGrid options
    'fullWidthRows': True,
    'syncColumnCellResize': True,
    'forceFitColumns': True,
    'defaultColumnWidth': 100,
    'rowHeight': 28,
    'enableColumnReorder': False,
    'enableTextSelectionOnCells': True,
    'editable': True,
    'autoEdit': False,
    'explicitInitialization': True,

    # Qgrid options
    'maxVisibleRows': 15,
    'minVisibleRows': 8,
    'sortable': False,
    'filterable': True,
    'highlightSelectedCell': False,
    'highlightSelectedRow': True
}

In [5]:
def read_in_cs_tables(scale_factor=6):
    
    h1 = r"H1_chemical_shift_table.jsn"
    c13 = r"C13_chemical_shift_table.jsn"
    
    H1df = pd.read_json(h1)
    C13df = pd.read_json(c13)
    
    H1df['meanCS']=(H1df.minCS+H1df.maxCS)/2
    H1df['sigmaCS']=(H1df.maxCS-H1df.minCS)/scale_factor
    
    C13df['meanCS']=(C13df.minCS+C13df.maxCS)/2
    C13df['sigmaCS']=(C13df.maxCS-C13df.minCS)/scale_factor
    
    for i in H1df.index:
        H1df.loc[i,'norm'] = stats.norm(loc=H1df.loc[i,'meanCS'], scale=H1df.loc[i,'sigmaCS'])

    for i in C13df.index:
        C13df.loc[i,'norm'] = stats.norm(loc=C13df.loc[i,'meanCS'], scale=C13df.loc[i,'sigmaCS'])
        
    return H1df, C13df

In [6]:
H1df_orig, C13df_orig = read_in_cs_tables()

In [7]:
# problems = []

# def on_upload_problemdir( change, problems):
    
#     files = change['new']
#     file_names = list(files.keys())
    
#     if len(file_names) == 0:
#         return
    
#     probdir_name, _ = file_names[0].split('.')
#     print(probdir_name)
    
    
    
#     with tempfile.TemporaryDirectory() as tmpdirname:

#         print('created temporary directory', tmpdirname)
#         problemDirectory = os.path.join(tmpdirname, probdir_name )
#         os.mkdir(os.path.join(tmpdirname, probdir_name))
        
#         for fname in files.keys():
#             if "yml" in fname:
#                 fp = open(os.path.join(problemDirectory, fname), 'wb')
#                 fp.write(files[fname]["content"])
#                 fp.close()
                
#             elif "pkl" in fname:
#                 fp = open(os.path.join(problemDirectory, fname), 'wb')
#                 fp.write(files[fname]["content"])
#                 fp.close()

#         print(problemDirectory)
#         problems.append(nmrProblem.NMRproblem(problemDirectory))
    
# upload_problemdir  = ipywidgets.widgets.FileUpload(multiple=True, 
#                                                   description="nmr problem",
#                                                   description_tooltip="choose all files in problem directory")


# upload_problemdir.observe(   lambda change: on_upload_problemdir(change, problems),  names='value')

In [8]:
# upload_problemdir

In [9]:
# problems[0].df

In [10]:
# #         #if 'procpar' in change['new'] and 'fid' in change['new']:
# #             try:
# # #                 print("read in nmr file")
# #                 procpar_file = change['new']['procpar']
# #                 fid_file     = change['new']['fid']

# #                 f, fprocpar =  tempfile.mkstemp( text=True) 

# #                 with open(fprocpar, "wb" ) as f:
# #                     f.write(procpar_file["content"])

# #                 f, ffid =  tempfile.mkstemp( text=True) 

# #                 with open(ffid, "wb" ) as f:
# #                     f.write(fid_file["content"])

# #                 nmrExpts[nmrExptKey] = nmrExpt.NMRexpt.from_varian(fid_file=ffid, procpar_file=fprocpar)
                
# #                 if nmrExpts[nmrExptKey].udic['ndim'] == 1:
                
# #                     nmrExpts[nmrExptKey].produce_1D(lb = [0.001])
                
# #                 else:
# #                     nmrExpts[nmrExptKey].produce_2D()

# with tempfile.TemporaryDirectory() as tmpdirname:
#     print('created temporary directory', tmpdirname)
#     problemDirectory = os.path.join(tmpdirname, "dummy_problem" )
#     os.mkdir(os.path.join(tmpdirname, "dummy_problem"))
    
    
#     fp = open(os.path.join(problemDirectory, "dummy_problem.yml"), 'wb')
#     yml_file = upload_problemdir.value['dummy_problem.yml']
#     fp.write(yml_file["content"])
#     fp.close()
    
#     fp = open(os.path.join(problemDirectory, "dummy_problem.pkl"), 'wb')
#     pkl_file = upload_problemdir.value['dummy_problem.pkl']
#     fp.write(pkl_file["content"])
#     fp.close()
#     print(os.listdir(tmpdirname))
#     print(os.listdir(problemDirectory))

    

#     print(problemDirectory)
#     nmrproblem = nmrProblem.NMRproblem(problemDirectory)

In [11]:
def update_attachedprotons_c13hyb(nmrproblem):
    iprobs = nmrproblem.iprobs
    udic = nmrproblem.udic
    hatoms = nmrproblem.protonAtoms
    catoms = nmrproblem.carbonAtoms
    df = nmrproblem.df


    for c in catoms:
        hsqc = df.loc['hsqc', c]
        attached_protons = 0
        for h in hsqc:
            attached_protons += int(df.loc['integral', h])

        df.loc['attached protons', c] = attached_protons
        df.loc['C13 hyb', c] = attached_protons

        for h in hsqc:
            df.loc['C13 hyb', h] = df.loc['C13 hyb', c]

In [12]:
def specwidthppm(udic, defaultvalue):
    sw = float(udic.get('sw', -1))
    if sw == -1:
        return defaultvalue
    else:
        return( sw / float(udic['obs']))
    
def tofppm(udic, defaultvalue):
    tof = float(udic.get('car', -1))
    if tof == -1:
        return defaultvalue
    else:
        return( tof / float(udic['obs']))

In [13]:
def updateSpectraWidgetsFromNMRproblem(ipwidgetsGUI):
    
    nmrproblem = ipwidgetsGUI.nmrproblem
#     global nmrproblem
    # protons
    try:
        protons = nmrproblem.udic[0]
        ipwidgetsGUI.pSpecWidthW.value = float(specwidthppm(protons, ipwidgetsGUI.pSpecWidthW.value)) 
        ipwidgetsGUI.pObsFreqW.value = float(protons.get('obs', ipwidgetsGUI.pObsFreqW.value))
        ipwidgetsGUI.pTofW.value = float(tofppm(protons, ipwidgetsGUI.pTofW.value)) 
        ipwidgetsGUI.pSizeW.value = int(protons.get('size', ipwidgetsGUI.pSizeW.value))
        ipwidgetsGUI.pLineBroadeningW.value = float(protons.get('lb', ipwidgetsGUI.pLineBroadeningW.value))
    except:
        pass
    
    # carbons
    try:
        carbons = nmrproblem.udic[1]
        ipwidgetsGUI.cSpecWidthW.value = float(specwidthppm(carbons, ipwidgetsGUI.cSpecWidthW.value)) 
        ipwidgetsGUI.cObsFreqW.value = float(carbons.get('obs', ipwidgetsGUI.cObsFreqW.value))
        ipwidgetsGUI.cTofW.value = float(tofppm(carbons, ipwidgetsGUI.cTofW.value)) 
        ipwidgetsGUI.cSizeW.value = int(carbons.get('size', ipwidgetsGUI.cSizeW.value))
        ipwidgetsGUI.cLineBroadeningW.value = float(carbons.get('lb', ipwidgetsGUI.cLineBroadeningW.value))
    except:
        pass

In [14]:
def updateMoleculeWidgetsFromNMRproblem(ipwidgetsGUI):
#     global nmrproblem
    nmrproblem = ipwidgetsGUI.nmrproblem
    ipwidgetsGUI.moleculeAtomsW.value = nmrproblem.moleculeAtomsStr
    ipwidgetsGUI.pGrpsW.value = nmrproblem.numProtonGroups
    ipwidgetsGUI.cGrpsW.value = nmrproblem.numCarbonGroups

In [15]:
def updateSpectralInformationWidgetChanged(ipwidgetsGUI):
#     global nmrproblem
    nmrproblem = ipwidgetsGUI.nmrproblem
    
# pSpecWidthW
# pObsFreqW
# pTofW
# pSizeW
# pLineBroadeningW

# ccSpecWidthW
# cObsFreqW
# cTofW
# cSizeW
# cLineBroadeningW  

    # check to see if udic exists and proton and carbon axes are there
    if not hasattr(nmrproblem, 'udic'):        
        return
    
    udic = nmrproblem.udic
    
    if 0 not in udic or 1 not in udic:
        return
    
    udic[0]['obs'] = ipwidgetsGUI.pObsFreqW.value
    udic[0]['sw'] = ipwidgetsGUI.pSpecWidthW.value * ipwidgetsGUI.pObsFreqW.value
    udic[0]['dw'] = 1.0 / (ipwidgetsGUI.pSpecWidthW.value * ipwidgetsGUI.pObsFreqW.value)
    udic[0]['car'] = ipwidgetsGUI.pTofW.value * ipwidgetsGUI.pObsFreqW.value
    udic[0]['size'] = ipwidgetsGUI.pSizeW.value
    udic[0]['lb'] = ipwidgetsGUI.pLineBroadeningW.value
    udic[0]['complex'] = True
    
    udic[1]['obs'] = ipwidgetsGUI.cObsFreqW.value
    udic[1]['sw'] = ipwidgetsGUI.cSpecWidthW.value * ipwidgetsGUI.cObsFreqW.value
    udic[1]['dw'] = 1.0 / (ipwidgetsGUI.cSpecWidthW.value * ipwidgetsGUI.cObsFreqW.value)
    udic[1]['car'] = ipwidgetsGUI.cTofW.value * ipwidgetsGUI.cObsFreqW.value
    udic[1]['size'] = ipwidgetsGUI.cSizeW.value
    udic[1]['lb'] = ipwidgetsGUI.cLineBroadeningW.value
    udic[1]['complex'] = True
        
    udic[0]['axis'] = ng.fileiobase.unit_conversion(udic[0]['size'],
                                                     udic[0]['complex'], 
                                                     udic[0]['sw'], 
                                                     udic[0]['obs'], 
                                                     udic[0]['car'])

    udic[1]['axis'] = ng.fileiobase.unit_conversion(udic[1]['size'],
                                                     udic[1]['complex'], 
                                                     udic[1]['sw'], 
                                                     udic[1]['obs'], 
                                                     udic[1]['car'])

In [16]:
# w1 = widgets.Output()


def display1H13C1Dspectr(ipwidgetsGUI, ax):
    
    nmrproblem = ipwidgetsGUI.nmrproblem
    
    udic = nmrproblem.udic
    
    full_spectrum1 = ax[0].plot(udic[1]['axis'].ppm_scale(), udic[1]['spec'], color='black', lw=0.5)
    ax[0].set_xlim(udic[1]['axis'].ppm_limits())
    full_spectrum2 = ax[1].plot(udic[0]['axis'].ppm_scale(), udic[0]['spec'], color='black', lw=0.5)
    ax[1].set_xlim(udic[0]['axis'].ppm_limits())
    # ax.set_xlim(10,-1)
    ax[0].spines['top'].set_visible(False)
    ax[0].spines['left'].set_visible(False)
    ax[0].spines['right'].set_visible(False)
    ax[0].set_xlabel('$^{13}$C [ppm]', fontsize=10, gid='c13ppm')
    ax[0].set_yticks([])

    ax[1].spines['top'].set_visible(False)
    ax[1].spines['left'].set_visible(False)
    ax[1].spines['right'].set_visible(False)
    ax[1].set_xlabel('$^{1}$H [ppm]', fontsize=10, gid='h1ppm')
    ax[1].set_yticks([])
    
    
def create1H13C1Doverlays(ipwidgetsGUI, ax):
    
    udic = ipwidgetsGUI.nmrproblem.udic

    peak_overlays = []
    peak_overlays_dict = {}

    for i in range(udic['ndim']):
        peak_overlays1 = []
        for Hi in udic[i]['info'].index:

            il = int(udic[i]['info'].loc[Hi,'pk_left'])
            ir = int(udic[i]['info'].loc[Hi,'pk_right'])


            pk, = ax[1-i].plot(udic[i]['axis'].ppm_scale()[il:ir], udic[i]['spec'][il:ir], 
                                  lw=0.5, c='black', label=Hi, gid=Hi)
            peak_overlays1.append(pk)
            peak_overlays_dict[Hi] = pk

        peak_overlays.append(peak_overlays1)
        
    cursor = mplcursors.cursor(peak_overlays[0]+peak_overlays[1], hover=True, highlight=True)    

    @cursor.connect("add")
    def on_add(sel):


        if str(sel.artist.get_label()) in udic[0]['atoms']:
            ii = 0
        else:
            ii = 1
        x,y = sel.target
        # print(x,y, udic[ii]['spec'].max())

        lbl = sel.artist.get_label()

        sel.extras[0].set_linewidth(0.75)
        sel.extras[0].set_color('red')
        
    return w1


def plotC13Distributions(ipwidgetsGUI, ax, numCandidates=5):
    
    ax.spines['top'].set_visible(False)
    ax.spines['left'].set_visible(False)
    ax.spines['right'].set_visible(False)
    ax.set_xlabel('ppm', fontsize=10, gid='ppm')
    ax.set_yticks([])
    
    # plot top three candidates for each carbon present
    #
    C13_ppm_axis = np.linspace(-30,250,500)
    carbon_atoms = ipwidgetsGUI.nmrproblem.carbonAtoms
    C13df = ipwidgetsGUI.nmrproblem.udic[1]['df']
    

    c13distdict = {}
    for k, ci in enumerate(carbon_atoms):
        distlist = []
        for i in ipwidgetsGUI.nmrproblem.iprobs[ci][:numCandidates]:
            c13distr, = ax.plot(C13_ppm_axis, C13df.loc[i,'norm'].pdf(C13_ppm_axis), 
                                label= C13df.loc[i, 'sF_latex_matplotlib'])
            
            c13distr.set_visible(False)
            distlist.append(c13distr)
            
        c13line  = ax.axvline(float(ipwidgetsGUI.nmrproblem.df.loc['ppm',ci]))
        c13line.set_visible(False)                         
        distlist.append(c13line)
        
        c13distdict[ci] = distlist
    
#         ax.legend();
#         ax.set_title("{} = {} ppm".format(ci,nmrproblem.df.loc['ppm',ci]))
    ax.set_xlabel('$^{13}$C [ppm]')
    ax.set_xlim(260,-40)
    
    return c13distdict

def plotH1Distributions(ipwidgetsGUI, ax, numCandidates=5):
    
    ax.spines['top'].set_visible(False)
    ax.spines['left'].set_visible(False)
    ax.spines['right'].set_visible(False)
    ax.set_xlabel('ppm', fontsize=10, gid='ppm')
    ax.set_yticks([])
    
    # plot top three candidates for each carbon present
    #
    H1_ppm_axis = np.linspace(-5,16,500)
    proton_atoms = ipwidgetsGUI.nmrproblem.protonAtoms
    H1df = ipwidgetsGUI.nmrproblem.udic[0]['df']
    

    h1distdict = {}
    for k, hi in enumerate(proton_atoms):
        distlist = []
        for i in ipwidgetsGUI.nmrproblem.iprobs[hi][:numCandidates]:
            h1distr, = ax.plot(H1_ppm_axis, H1df.loc[i,'norm'].pdf(H1_ppm_axis), 
                                label= H1df.loc[i, 'sF_latex_matplotlib'])
            
            h1distr.set_visible(False)
            distlist.append(h1distr)
            
        h1line  = ax.axvline(float(ipwidgetsGUI.nmrproblem.df.loc['ppm',hi]))
        h1line.set_visible(False)                         
        distlist.append(h1line)
        
        h1distdict[hi] = distlist
    
#         ax.legend();
#         ax.set_title("{} = {} ppm".format(hi,nmrproblem.df.loc['ppm',hi]))
    ax.set_xlabel('$^{1}$H [ppm]')
    ax.set_xlim(12,-2)
    
    return h1distdict
        


    
def createH1C13interactivePlot(ipwidgetsGUI): 
    
    w1 = widgets.Output()

    try:
        udic = ipwidgetsGUI.nmrproblem.udic

        # delay plotting by using with statement
        # place matplotlib plot into a ipywidget
        with w1:
            # create four plots using a grid layout
            fig = plt.figure(constrained_layout=True, figsize=(9,7))

            gs = GridSpec(2, 3, figure=fig)
            ax1 = fig.add_subplot(gs[0, :-1])
            # identical to ax1 = plt.subplot(gs.new_subplotspec((0, 0), colspan=3))
            ax2 = fig.add_subplot(gs[1, :-1])
            ax3 = fig.add_subplot(gs[0, -1])
            ax4 = fig.add_subplot(gs[1, -1])

            # put the axes into an array for easier axis
            ax = [ax1,ax2,ax3,ax4]

        # create 1D plots 1H and 13C in ax1 and ax2 respectively
        # carbon on top, proton on bottom
        display1H13C1Dspectr(ipwidgetsGUI, ax)

        # plot all the distributions in iprobs and return each plot that belongs
        # to a specific label, C1, C2 .. in a dictionary of lists
        # the keys are the labels C1, C2, C3 or h1, H2, H3 ...
        # the values are a list of the indexes of the main df table that correspond to the peak based
        # on chemical shift, dbe, integrals
        
        c13distdict = plotC13Distributions(ipwidgetsGUI, ax3, numCandidates=3)
        h1distdict = plotH1Distributions(ipwidgetsGUI, ax4, numCandidates=3)

        # the dictionaries are put in a list so that they can be easier indexed
        h1c13distlist = [c13distdict, h1distdict]

        #  plot the peak overlays on top of the 1D 1H and 13C spectra
        # keep account of the different overlays by storing them in a dictionary of lists
        peak_overlays = []
        peak_overlays_dict = {}

        # for proton and carbon spectra
        for i in range(udic['ndim']):
            peak_overlays1 = []
            for Hi in udic[i]['info'].index:

                il = int(udic[i]['info'].loc[Hi,'pk_left'])
                ir = int(udic[i]['info'].loc[Hi,'pk_right'])


                pk, = ax[1-i].plot(udic[i]['axis'].ppm_scale()[il:ir], udic[i]['spec'][il:ir], 
                                      lw=0.5, c='black', label=Hi, gid=Hi)
                peak_overlays1.append(pk)
                peak_overlays_dict[Hi] = pk

            peak_overlays.append(peak_overlays1)

        # connect the overlays to the cursor
        cursor = mplcursors.cursor(peak_overlays[0]+peak_overlays[1], hover=True, highlight=True)    

        # function to add the labels when the cursor hovers over a resonance.
        # cursor can be over carbon or proton 1D spectrum
        # if carbon proton connectivity is known via hsqc information then the coresponding
        # peak in the opposite spectrum is highlighed.
        # the probability distributions for the highlighted peak
        # are shown in the carbon and proton distribution plots
        @cursor.connect("add")
        def on_add(sel):

            if str(sel.artist.get_label()) in udic[0]['atoms']:
                ii = 0
            else:
                ii = 1
            x,y = sel.target
            # print(x,y, udic[ii]['spec'].max())

            # use artist to labal to find out peak id, H1, H2, ... or C1, C2, ...
            lbl = sel.artist.get_label()
            selected_pk = str(lbl)

            # set selected peak to red with a linewidth of 0.75
            sel.extras[0].set_linewidth(0.75)
            sel.extras[0].set_color('red')

            # figure out if selected peak is a proton or a carbon
            if selected_pk in udic[0]['atoms']:
                ii=0
            else:
                ii=1           

            # change selection text depending on what hight peak was picked
            # top, middle, bottom
            # pk_y is an index position from 0,1,2
            pk_y = 2-int(y*100/udic[ii]['info'].loc[lbl,'pk_y'])//35

            # set annotation text
            sel.annotation.set_text(udic[ii]['labels1_dict'][selected_pk][pk_y])

            # highlight coresponding proton or carbon peaks
            # from hsqc data find corresponding peaks
            highlighted_pks = udic[ii]['info'].loc[selected_pk,'hsqc']        
            for hpk in highlighted_pks:
                sel.extras.append(cursor.add_highlight(peak_overlays_dict[hpk]))
                sel.extras[-1].set_linewidth(0.75)
                sel.extras[-1].set_color('red')

            # add highlighted distributions

            colors = ['b', 'g', 'r', 'c', 'm', 'y', 'k']

            # used to dsplay legends of highlighted distributions
            plines1 = []
            plabels1 = []

            # set visible
            # circle through colors
            for hpk in highlighted_pks:
                for i,aa in enumerate(h1c13distlist[ii][hpk]):
                    plines1.append(aa)
                    plabels1.append(aa.get_label())

                    sel.extras.append(cursor.add_highlight(aa))
                    sel.extras[-1].set_visible(True)
                    sel.extras[-1].set_linewidth(0.75)
                    sel.extras[-1].set_color(colors[i%7])

            # depending whether highlighted peak is carbon or proton
            # choose correct axis
            if ii == 0:
                ax[2].legend(plines1, plabels1) 
            else:
                ax[3].legend(plines1, plabels1) 

            # flip index for selected pk
            ii2 = 0                          
            if ii == 0:
                ii2 = 1

            # reset lists for legends
            plines1 = []
            plabels1 = []
            
            # set visible
            # circle through colors
            for i,aa in enumerate(h1c13distlist[ii2][selected_pk]):

                plines1.append(aa)
                plabels1.append(aa.get_label())

                sel.extras.append(cursor.add_highlight(aa))
                sel.extras[-1].set_visible(True)
                sel.extras[-1].set_linewidth(0.75)
                sel.extras[-1].set_color(colors[i%3])

            # depending whether highlighted peak is carbon or proton
            # choose correct axis
            if ii == 0:
                ax[3].legend(plines1, plabels1) 
            else:
                ax[2].legend(plines1, plabels1) 

        return w1 # if everything works return plot in an ipywidget
    except:
        print("Exception creating plotting window")
        return widgets.Output() # if not return empyt output ipywidget


In [17]:
# widget to hold pandas dataframe of NMR imformation extracted
# from spectra. The widget is based on a qgrid widget and held in a VBOX
# so that we can update the widget when a dataset is read in or the molecule
# has been changed or a new problem is started.

# dfWarningTextW = widgets.Label("Table Messages:  OK")
# dfUpdateTableB = widgets.Button(description="update table")
# dfRunAnalysisB = widgets.Button(description="update and run")

# dfButtonsLayout = widgets.HBox([dfUpdateTableB,dfRunAnalysisB])
# dfLayout = widgets.VBox([dfWarningTextW])  # start with an empty widget
# dfLayout



class ipywidgetsDisplay(widgets.Tab):
    
    def __init__(self, nmrproblem=None):
        
        super().__init__()
        
        if not isinstance(nmrproblem, nmrProblem.NMRproblem):
            
            self.nmrproblem = nmrproblem
            self.df = pd.DataFrame()
            
        else:
            self.nmrproblem = nmrproblem
            self.df = nmrproblem.df
            
        # create save problem widgets
        self.saveProblemButtonW = widgets.Button(description="Save Problem")

        # widgets to obtain problem working directory
        self.prDirW = widgets.Text(value='',
                                 placeholder='problem directory',
                                 description='problem directory',
                                 disabled=False)

        self.prDirB = widgets.Button(description='Set Directory')
#         self.prDirLayout = widgets.HBox([self.prDirW, self.prDirB])
        
    
        self.upload_problemdir  = ipywidgets.widgets.FileUpload(multiple=True, 
                                                                  description="nmr problem",
                                                                  description_tooltip="choose all files in problem directory")

        self.prDirLayout = widgets.HBox([ self.upload_problemdir])
        # widgets to obtain info on the molecule
        # number and tye of atoms in molecule
        # number of proton resonances in molecule
        # number of carbon resonance in molecule
        self.moleculeAtomsW = widgets.Text(value='',
                                           placeholder='atoms in molecule',
                                           description='atoms',
                                           disabled=False)

        self.pGrpsW = widgets.IntText(value=1,
                                      placeholder='H1 groups in spectrum',
                                      description='H1 groups',
                                      disabled=False)

        self.cGrpsW = widgets.IntText(value=1,
                                      description='C13 groups',
                                      disabled=False)

        self.moleculesSubmitB = widgets.Button(description="Update Molecule")

        self.moleculeLayout = widgets.VBox([self.moleculeAtomsW, self.pGrpsW, self.cGrpsW, self.moleculesSubmitB])
        
        # widgets to set 1D spectral parameters for proton and carbon
        self.pLabelW = widgets.Label("$^{1}H$")

        self.pSpecWidthW = widgets.FloatText(value=12.0,
                                             tooltip='proton spectral width',
                                             description='sw (ppm)',
                                             disabled=False)    

        self.pObsFreqW = widgets.FloatText(value=400.0,
                                           description='obs (MHz)',
                                           disabled=False)

        self.pTofW = widgets.FloatText(value=5.0,
                                       description='tof (ppm)',
                                       diabled=False)

        self.pSizeW = widgets.IntText(value=32768,
                                      description='size (pts)',
                                      disabled=False)

        self.pLineBroadeningW = widgets.FloatText(value=0.5,
                                                  description='lb (Hz)',
                                                  disabled=False)

        self.cLabelW = widgets.Label("$^{13}C$")

        self.cSpecWidthW = widgets.FloatText(value=210.0,
                                             description='sw (ppm)',
                                             disabled=False)    

        self.cObsFreqW = widgets.FloatText(value=100.0,
                                           description='obs (MHz)',
                                           disabled=False)

        self.cTofW = widgets.FloatText(value=5.0,
                                       description='tof (ppm)',
                                       diabled=False)

        self.cSizeW = widgets.IntText(value=32768,
                                      description='size (pts)',
                                      disabled=False)

        self.cLineBroadeningW = widgets.FloatText(value=0.5,
                                                  description='lb (Hz)',
                                                  disabled=False)

        self.specSubmitB = widgets.Button(description="Update Spectra")

        self.specLayout = widgets.HBox( [ widgets.VBox([self.pLabelW, self.pObsFreqW, self.pSpecWidthW, 
                                                        self.pTofW, self.pSizeW, self.pLineBroadeningW, 
                                                        self.specSubmitB]),
                                           widgets.VBox([self.cLabelW, self.cObsFreqW, self.cSpecWidthW, self.cTofW, 
                                                         self.cSizeW, self.cLineBroadeningW])])


        self.old = 'All'
        self.new = 'ALL'
        
        self.toggleDF = widgets.ToggleButtons(  options=['All', 'integrals-ppm','COSY', 'HSQC-HMBC'],
                                                description='Display:',
                                                disabled=False,
                                                button_style='', # 'success', 'info', 'warning', 'danger' or ''
                                                tooltips=['Show full Dataframe', 'Show COSY Input', 'Show HSQC/HMBC Input'],
                                            #     icons=['check'] * 3
                                             )
        
        self.qgrid1 = qgrid.show_grid(self.df,  grid_options=qgrid_opts)
        
        self.toggleDF.observe(self.toggleValue)
        
        self.dfWarningTextW = widgets.Label("Table Messages:  OK")
        self.dfUpdateTableB = widgets.Button(description="update table")
        self.dfRunAnalysisB = widgets.Button(description="update and run")
        
        self.dfButtonsLayout = widgets.HBox([self.dfUpdateTableB, self.dfRunAnalysisB])
        self.dfLayout = widgets.VBox([self.toggleDF, 
                                      self.dfWarningTextW,
                                      self.qgrid1,
                                      self.dfButtonsLayout])


        self.accordion = widgets.Accordion(children=[self.prDirLayout, 
                                                     self.moleculeLayout, 
                                                     self.specLayout, 
                                                     self.dfLayout])
        self.accordion.set_title(0, "Problem Directory")
        self.accordion.set_title(1, "Molecule")
        self.accordion.set_title(2, "Spectroscopy")
        self.accordion.set_title(3, "DataSet")
        self.page1 = widgets.VBox([self.accordion,self.saveProblemButtonW])
        
        self.H1C131DplotsLayout = widgets.VBox([widgets.Output()])

        self.children = [self.page1,  self.H1C131DplotsLayout]
        self.set_title(0,'Problem Setup')
        self.set_title(1, 'Problem Plots')
        
        
#         self.prDirB.on_click(self.onButtonClicked)
        self.upload_problemdir.observe(   lambda change: self.on_upload_problemdir(change),  names='value')
        self.moleculesSubmitB.on_click(self.onButtonClicked)
        self.specSubmitB.on_click(self.onButtonClicked)
        self.dfUpdateTableB.on_click(self.onButtonClicked)
        self.dfRunAnalysisB.on_click(self.onButtonClicked)
        self.saveProblemButtonW.on_click(self.onButtonClicked)
        
        
    def on_upload_problemdir( self, change):

        files = change['new']
        file_names = list(files.keys())

        if len(file_names) == 0:
            return

        probdir_name, _ = file_names[0].split('.')
        print(probdir_name)



        with tempfile.TemporaryDirectory() as tmpdirname:

            print('created temporary directory', tmpdirname)
            problemDirectory = os.path.join(tmpdirname, probdir_name )
            os.mkdir(os.path.join(tmpdirname, probdir_name))

            for fname in files.keys():
                if "yml" in fname:
                    fp = open(os.path.join(problemDirectory, fname), 'wb')
                    fp.write(files[fname]["content"])
                    fp.close()

                elif "pkl" in fname:
                    fp = open(os.path.join(problemDirectory, fname), 'wb')
                    fp.write(files[fname]["content"])
                    fp.close()

            print(problemDirectory)
            self.nmrproblem = nmrProblem.NMRproblem(problemDirectory)
            
            if not isinstance( self.nmrproblem, type(None)):
                self.prDirW.value = self.nmrproblem.problemDirectoryPath

                # update other widgets based on contents of nmrproblem
                updateSpectraWidgetsFromNMRproblem(self)
                updateMoleculeWidgetsFromNMRproblem(self)

                # create a view of the dataframe in nmrproblem
                self.qgrid1.df = self.nmrproblem.df
                self.dfWarningTextW.value = "Table Messages: None"

#                 dfLayout.children = [dfWarningTextW, qgrid1, dfButtonsLayout]

                # create 1D 1H & C13 plots widget
                self.H1C131DplotsLayout.children = [ createH1C13interactivePlot(self) ]


    def toggleValue(self, event):
        
        if self.qgrid1.get_changed_df().shape == (0,0):
            return
    #     print(event)
        if event.name == 'value':
            
            self.old = event.old
            self.new = event.new
            # update backend dataframe
            
            # i
            if event.old == 'All':
                self.nmrproblem.df.loc[:,:] =  self.qgrid1.get_changed_df().loc[:,:]
                
            elif event.old == 'COSY':
                rr = ['ppm'] + self.nmrproblem.protonAtoms
                cc = ['ppm'] + self.nmrproblem.protonAtoms
                self.nmrproblem.df.loc[rr, cc] =  self.qgrid1.get_changed_df().loc[rr, cc]
                
            elif event.old == 'HSQC-HMBC':
                rr = ['ppm'] + self.nmrproblem.carbonAtoms
                cc = ['ppm'] + self.nmrproblem.protonAtoms
                self.nmrproblem.df.loc[rr, cc] = self.qgrid1.get_changed_df().loc[rr, cc]
                
            elif event.old == 'integrals-ppm':
                rr = ['ppm', 'integral']
                cc = self.nmrproblem.protonAtoms + self.nmrproblem.carbonAtoms
                
                self.nmrproblem.df.loc[rr, cc] = self.qgrid1.get_changed_df().loc[rr, cc]
                                
            # update qgrid display of dataframe
            if event.new == 'All':
                self.qgrid1.df = self.nmrproblem.df
                
            elif event.new == 'COSY':
                rr = ['ppm'] + self.nmrproblem.protonAtoms[::-1]
                cc = ['ppm'] + self.nmrproblem.protonAtoms
                self.qgrid1.df = self.nmrproblem.df.loc[rr, cc]
                
            elif event.new == 'HSQC-HMBC':
                rr = ['ppm'] + self.nmrproblem.carbonAtoms[::-1]
                cc = ['ppm'] + self.nmrproblem.protonAtoms
                self.qgrid1.df = self.nmrproblem.df.loc[rr, cc]
                
            elif event.new == 'integrals-ppm':
                rr = ['ppm', 'integral']
                cc = self.nmrproblem.protonAtoms + self.nmrproblem.carbonAtoms
                
                self.qgrid1.df = self.nmrproblem.df.loc[rr, cc]
                

    def onButtonClicked(self,bttn):

    #     print("bttn", bttn, type(bttn), bttn.description)
        if "Directory" in bttn.description:
    #         print("Directory")
            self.nmrproblem = nmrProblem.NMRproblem.from_guidata()
    #         print(type(nmrproblem))
            if not isinstance( self.nmrproblem, type(None)):
                self.prDirW.value = self.nmrproblem.problemDirectoryPath

                # update other widgets based on contents of nmrproblem
                updateSpectraWidgetsFromNMRproblem(self)
                updateMoleculeWidgetsFromNMRproblem(self)

                # create a view of the dataframe in nmrproblem
                self.qgrid1.df = self.nmrproblem.df
                self.dfWarningTextW.value = "Table Messages: None"

#                 dfLayout.children = [dfWarningTextW, qgrid1, dfButtonsLayout]

                # create 1D 1H & C13 plots widget
                self.H1C131DplotsLayout.children = [ createH1C13interactivePlot(self) ]

        elif "update table" in bttn.description:
            print("update table")
            ipwidgetsGUI.toggleDF.value='All'
            ok = self.nmrproblem.updateDFtable( self.qgrid1.get_changed_df())

            print('ok',ok)

            if ok:
                nmrProblem.convertHSQCHMBCCOSYtoLists(self.nmrproblem)
                update_attachedprotons_c13hyb(self.nmrproblem)
                self.qgrid1.df = self.nmrproblem.df
                self.dfWarningTextW.value = "Table Messages: None"
            else:
                 self.dfWarningTextW.value = "Table Messages: problems in table, please check it"

        elif "update and run" in bttn.description:
            
            ipwidgetsGUI.toggleDF.value='All'

            ok = self.nmrproblem.updateDFtable( self.qgrid1.get_changed_df())

            if ok:
                self.qgrid1.df = self.nmrproblem.df
                self.dfWarningTextW.value = "Table Messages: None"

                self.nmrproblem.dfToNumbers()
                nmrProblem.convertHSQCHMBCCOSYtoLists(self.nmrproblem)
                nmrProblem.convertJHzToLists(self.nmrproblem)
                
                update_attachedprotons_c13hyb(self.nmrproblem)

#                 H1df_orig, C13df_orig = nmrProblem.readinChemShiftTables()
                H1df_orig, C13df_orig = read_in_cs_tables()

                H1df, C13df, probDistFunctions = nmrProblem.calcProbDistFunctions(self.nmrproblem, H1df_orig, C13df_orig)

                iprobs = nmrProblem.identify1HC13peaks(self.nmrproblem, H1df, C13df )

                self.nmrproblem.udic[0]['df'] = H1df
                self.nmrproblem.udic[1]['df'] = C13df

                self.nmrproblem.iprobs = iprobs

                self.nmrproblem.createInfoDataframes()

                nmrProblem.calculate1H13CSpectra1D(self.nmrproblem)

                udic = self.nmrproblem.udic

                nmrProblem.save1DspecInfotoUdic(self.nmrproblem)

                nmrProblem.create1H13Clabels(self.nmrproblem, iprobs, num_poss=3)

    #             plotC13Distributions(nmrproblem)
    #             plotH1Distributions(nmrproblem, numCandidates=5)



                # create 1D 1H & C13 plots widget

    #             H1C131DplotsLayout.children = [ nmrProblem.create1D_H1C13plot(nmrproblem) ]
                self.H1C131DplotsLayout.children = [ createH1C13interactivePlot(self) ]
            else:
                self.dfWarningTextW.value = "Table Messages: problems in table, please check it"


        elif "Spectra" in bttn.description:
    #         print("Spectra")
            self.nmrproblem.createInfoDataframes()
            udic = self.nmrproblem.udic
            nmrProblem.save1DspecInfotoUdic(self.nmrproblem)
    #         nmrProblem.create1H13Clabels(nmrproblem, iprobs, num_poss=3)

            updateSpectralInformationWidgetChanged(self)
            nmrProblem.calculate1H13CSpectra1D(nmrproblem)
            self.H1C131DplotsLayout.children = [ createH1C13interactivePlot(self) ]

        elif "Molecule" in bttn.description:
            print("Molecule")

            self.nmrproblem.update_molecule_ipywidgets(self.moleculeAtomsW.value,
                                                       self.pGrpsW.value,
                                                       self.cGrpsW.value)

            # create a view of the dataframe in nmrproblem
            self.qgrid1.df = self.nmrproblem.df
            self.dfWarningTextW.value = "Table Messages: None"

#             self.dfLayout.children = [dfWarningTextW, qgrid1, dfButtonsLayout]

        elif "Save Problem" in bttn.description:
            self.nmrproblem.save_as_yml()

            fn = os.path.join(self.nmrproblem.rootDirectory,
                              self.nmrproblem.problemDirectory,  
                              self.nmrproblem.problemDirectory + '.pkl')
            fp = open(fn, 'wb')
            pickle.dump(self.nmrproblem.udic, fp)
            fp.close()



            

In [18]:
ipwidgetsGUI = ipywidgetsDisplay()
ipwidgetsGUI

ipywidgetsDisplay(children=(VBox(children=(Accordion(children=(HBox(children=(FileUpload(value={}, description…

dummy_problem
created temporary directory C:\Users\ERIC\AppData\Local\Temp\tmp8vb33f8o
C:\Users\ERIC\AppData\Local\Temp\tmp8vb33f8o\dummy_problem
['dummy_problem.pkl']
read un pickle data


AttributeError: 'NoneType' object has no attribute 'df'